In [ ]:
!nvidia-smi

Thu Oct 17 12:14:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install accelerate
!pip install transformers==4.45.2
!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy2
!pip install peft
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/RefalMachine/llmtf_open
%cd llmtf_open
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl

Cloning into 'llmtf_open'...
remote: Enumerating objects: 504, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 504 (delta 53), reused 56 (delta 29), pack-reused 420 (from 1)
Receiving objects: 100% (504/504), 1.18 MiB | 3.21 MiB/s, done.
Resolving deltas: 100% (338/338), done.
/content/llmtf_open
--2024-10-17 12:16:03--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291979 (1.2M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   1.23M  --.-KB/s    in 0.03s   

2024-10-17 12:16:03 (36.5 MB/s) - ‘train.jsonl’ saved [1291979/1291979]



In [ ]:
!ls

conversation_configs  evaluate_model.py  requirements.txt		      todo.txt
Dockerfile	      examples		 run_evaluate_multinode_multigpu.py   train.jsonl
eval_grammar.py       llmtf		 run_evaluate_multinode_multigpu.sh
eval_RuOpinionNE.py   README.md		 run_evaluate_singlenode_multigpu.sh


In [ ]:
from llmtf.tasks.ruopinionne import RuOpinionNE, pred2opinions_default
instruction = '''Твоя задача состоит в том, чтобы проанализировать текст и извлечь из него выражения мнений, представленные в виде кортежа мнений, состоящих из 4 основных составляющих:
1. Источник мнения: автор, именованная сущность текста (подстрока исходного текста), либо NULL. Key = Source;
2. Объект мнения: именованная сущность в тексте (подстрока исходного текста). Key = Target;
3. Тональность: положительная/негативная (POS/NEG). Key = Polarity;
4. Языковое выражение: аргумент, на основании которого принята результирующая тональность (одна или несколько подстрок исходного текста). Key = Expression;

Если источник мнения отсутствуют, то Source = NULL, если автор является источником мнения, то AUTHOR. Ответ необходимо представить в виде json списка, каждый элемент которого является кортежем мнений. Каждый кортеж мнений это словарь, состоящий из четырех значений: Source, Target, Polarity, Expression.
***Текст***
{text}'''

task = RuOpinionNE(instruction=instruction, pred2opinion=pred2opinions_default)

In [ ]:
from llmtf.model import HFModel

model_name_or_path = 'Qwen/Qwen2.5-3B-Instruct'
model = HFModel(conversation_template_path='conversation_configs/qwen2.json', device_map='cuda:0', attn_implementation="sdpa")
model.from_pretrained(model_name_or_path)

model.generation_config.max_new_tokens = 200
model.generation_config.repetition_penalty = 1.0
model.generation_config.do_sample = False
model.generation_config.temperature = 0.0
model.generation_config

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

INFO: 2024-10-17 12:17:30,927: llmtf.base.hfmodel: Set eos_token_id in generation_config to [151645]
INFO:llmtf.base.hfmodel:Set eos_token_id in generation_config to [151645]
INFO: 2024-10-17 12:17:30,932: llmtf.base.hfmodel: Model id: Qwen/Qwen2.5-3B-Instruct
INFO:llmtf.base.hfmodel:Model id: Qwen/Qwen2.5-3B-Instruct
INFO: 2024-10-17 12:17:30,934: llmtf.base.hfmodel: Leading space: False
INFO:llmtf.base.hfmodel:Leading space: False


GenerationConfig {
  "eos_token_id": [
    151645
  ],
  "max_length": 32768,
  "max_new_tokens": 200,
  "pad_token_id": 151643,
  "stop_strings": [
    "<|im_end|>"
  ],
  "temperature": 0.0,
  "top_k": 40,
  "top_p": 0.9,
  "trust_remote_code": false
}

In [ ]:
!cat conversation_configs/qwen2.json

{
    "system_prompt": "",
    "system_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
    "user_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
    "bot_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
    "bot_message_template_incomplete": "<|im_start|>{role}\n{content}",
    "user_role": "user",
    "bot_role": "assistant",
    "system_role": "system",
    "global_prefix": "",
    "suffix": "<|im_start|>assistant\n",
    "add_special_tokens": false,
    "eos_token": "<|im_end|>"
}


In [ ]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./RuOpinionNE_test1',
    max_len=4000,
    few_shot_count=5,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=20 # итоговую оценку нужно прислать для 200 сэмплов
)

INFO: 2024-10-17 12:17:31,063: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [151645]
INFO:llmtf.base.hfmodel:Updated generation_config.eos_token_id: [151645]
INFO: 2024-10-17 12:17:31,066: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
INFO:llmtf.base.hfmodel:Updated generation_config.stop_strings: ['<|im_end|>']
100%|██████████| 20/20 [00:00<00:00, 41.91it/s]
INFO: 2024-10-17 12:17:31,760: llmtf.base.ruopinionne: Loading Dataset: 0.69s
INFO:llmtf.base.ruopinionne:Loading Dataset: 0.69s
  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is 

Fail to validate
{'Source': 'NULL', 'Target': 'парламента', 'Polarity': 'NEG', 'Expression': 'роспуском'}
Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов.
Fail to validate
{'Source': 'NULL', 'Target': 'такой вариант', 'Polarity': 'NEU', 'Expression': 'маловероятным'}
Несмотря на то, что такой вариант является маловероятным, итальянские политологи рассматривают его в качестве возможного исхода ситуации.


 60%|██████    | 3/5 [02:12<01:32, 46.31s/it]

Fail to validate
{'Source': 'NULL', 'Target': 'журналистов', 'Polarity': 'NEG', 'Expression': 'отставки Гречмана'}
Отставки Гречмана потребовали томские журналисты после жестокого избиения их коллеги Константина Попова, совершенного милиционером.


 80%|████████  | 4/5 [03:00<00:46, 46.98s/it]

Fail to validate
{'Source': 'Оппозиция', 'Target': 'администрацию Руссефф', 'Polarity': 'NEG', 'Expression': 'нарушила закон, перераспределяя средства из бюджета с целью обеспечить себе общественную поддержку несмотря на слабеющую экономику'}
Оппозиция утверждает, что администрация Руссефф нарушила закон, перераспределяя средства из бюджета с целью обеспечить себе общественную поддержку несмотря на слабеющую экономику в преддверии президентских выборов 2014 года.


100%|██████████| 5/5 [03:41<00:00, 44.32s/it]
INFO: 2024-10-17 12:21:13,384: llmtf.base.ruopinionne: Processing Dataset: 221.62s
INFO:llmtf.base.ruopinionne:Processing Dataset: 221.62s
INFO: 2024-10-17 12:21:13,388: llmtf.base.ruopinionne: Results for ruopinionne:
INFO:llmtf.base.ruopinionne:Results for ruopinionne:
INFO: 2024-10-17 12:21:13,393: llmtf.base.ruopinionne: {'f1': 0.14041095890410962}
INFO:llmtf.base.ruopinionne:{'f1': 0.14041095890410962}


Fail to validate
{'Source': 'NULL', 'Target': 'Тилль Линдеманна', 'Polarity': 'NEG', 'Expression': 'с огнемётом на сцене'}
Тилль Линдеманн с огнемётом на сцене.


In [ ]:
!ls ./RuOpinionNE_test1/

ruopinionne.jsonl  ruopinionne_params.jsonl  ruopinionne_total.jsonl


In [ ]:
!cat ./RuOpinionNE_test1/ruopinionne_total.jsonl

{
    "task_name": "ruopinionne",
    "results": {
        "f1": 0.14041095890410962
    },
    "leaderboard_result": 0.14041095890410962
}


In [ ]:
!cat ./RuOpinionNE_test1/ruopinionne_params.jsonl

{
    "custom_generation_config": null,
    "model_params": {
        "model_name_or_path": "Qwen/Qwen2.5-3B-Instruct",
        "generation_config": {
            "eos_token_id": [
                151645
            ],
            "max_length": 32768,
            "max_new_tokens": 200,
            "pad_token_id": 151643,
            "stop_strings": [
                "<|im_end|>"
            ],
            "temperature": 0.0,
            "top_k": 40,
            "top_p": 0.9,
            "transformers_version": "4.45.2",
            "trust_remote_code": false
        },
        "conversation_template": {
            "system_prompt": "",
            "system_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
            "user_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
            "bot_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
            "bot_message_template_incomplete": "<|im_start|>{role}\n{content}",
            "user_role": "use

In [ ]:
!cat ./RuOpinionNE_test1/ruopinionne.jsonl

{
    "metric": {
        "f1": {
            "gold": {
                "sent_id": 23,
                "text": "Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов.",
                "opinions": []
            },
            "preds": {
                "opinions": [],
                "sent_id": 23,
                "text": "Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов."
            }
        }
    },
    "predict": "***\n[\n    {\n        \"Source\": \"NULL\",\n        \"Target\": \"парламента\",\n        \"Polarity\": \"NEG\",\n        \"Expression\": \"роспуском\"\n    }\n]",
    "sample": {
        "sent_id": 23,
        "text": "Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов.",
        "opinions": []
    },
    "prompt": "<|im_start|>user\nТвоя задача состоит в том, чтобы проанализировать текст и извлечь из